## [Example: Constant + MuE (Profile HMM)](http://pyro.ai/examples/mue_profile.html#example-constant-mue-profile-hmm)

### A standard profile HMM model [1], which corresponds to a constant (delta function) distribution with a MuE observation [2]. 
#### This is a standard generative model of variable-length biological sequences (e.g. proteins) which does not require preprocessing the data by building a multiple sequence alignment (MSA). It can be compared to a more complex MuE model in this package, the FactorMuE.

### References:
[1] R. Durbin, S. R. Eddy, A. Krogh, and G. Mitchison (1998)
"Biological sequence analysis: probabilistic models of proteins and nucleic
acids"
Cambridge university press

[2] E. N. Weinstein, D. S. Marks (2021)
"A structured observation distribution for generative biological sequence
prediction and forecasting"
https://www.biorxiv.org/content/10.1101/2020.07.31.231381v2.full.pdf

In [ ]:
%%sh
curl -O https://raw.githubusercontent.com/debbiemarkslab/MuE/master/models/examples/ve6_full.fasta

[Data](https://github.com/debbiemarkslab/MuE/blob/master/models/examples/ve6_full.fasta)

In [1]:
import argparse
import datetime
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.optim import Adam

import pyro
from pyro.contrib.mue.dataloaders import BiosequenceDataset
from pyro.contrib.mue.models import ProfileHMM
from pyro.optim import MultiStepLR

[MuE](https://github.com/pyro-ppl/pyro/tree/dev/pyro/contrib/mue)

In [2]:
file = './ve6_full.fasta'

In [3]:
seqs = []
seq = ""
with open(file, 'r') as fr:
    for line in fr:
        if line[0] == '>':
           if seq !="":
                seq += "*"
                seqs.append(seq)
                seq=""
        else:
            seq += line.strip('\n')
 

In [4]:
len(seqs)

1608

In [5]:
dataset = BiosequenceDataset(
    file,
    'fasta',
    alphabet= 'amino-acid',
    include_stop=True,
    device='cpu'

)

In [6]:
dataset.data_size

1609

In [7]:
dataset.alphabet_length

21

In [8]:
dataset.max_length

159

In [9]:
len(dataset)

1609

In [10]:
dataset.seq_data

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [11]:
batch_size = 10
split = 0.2

In [12]:
holdout_num = int(np.ceil(split*len(dataset)))

In [13]:
holdout_num

322

In [14]:
data_lengths = [len(dataset)-holdout_num, holdout_num]

In [15]:
data_lengths

[1287, 322]

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [17]:
indices = torch.randperm(sum(data_lengths), device=device).tolist()

In [18]:
for t in torch._utils._accumulate(data_lengths):
    print(t)

1287
1609


In [19]:
dataset_train, dataset_test = [
    torch.utils.data.Subset(dataset, indices[(offset-length):offset])
    for offset, length in zip(
        torch._utils._accumulate(data_lengths),
        data_lengths
    )
]

In [20]:
latent_seq_length = int(dataset.max_length * 1.1)
latent_seq_length

174

In [21]:
model = ProfileHMM(
    latent_seq_length=latent_seq_length,
    alphabet_length= dataset.alphabet_length,
    prior_scale=1.0,
    cuda=False,
    indel_prior_bias= 10.0,
    pin_memory=False
)

In [22]:
scheduler = MultiStepLR(
    {
        'optimizer': Adam,
        'optim_args': {'lr':0.001},
        'milestones': json.loads("[]"),
        'gamma': 0.5
    }
)

In [23]:
n_epochs = 10

In [31]:
torch.set_default_dtype(torch.float64)

In [32]:
losses = model.fit_svi(dataset, 
                        n_epochs, 
                        batch_size, 
                        scheduler, 
                        jit= False
                    )

RuntimeError: expected scalar type Float but found Double
     Trace Shapes:           
      Param Sites:           
     Sample Sites:           
precursor_seq dist | 174 21  
             value | 174 21  
   insert_seq dist | 175 21  
             value | 175 21  
       insert dist | 174  3 2
             value | 174  3 2
       delete dist | 174  3 2
             value | 174  3 2